In [3]:
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

In [4]:
# 1. 데이터셋 (간단 예시)
texts = [
    "Box box box",
    "My tyres are gone",
    "Tell him to get out",
    "We need to push",
    "Engine is overheating",
    "Good job keep pushing",
    "I'm losing power",
    "Switching to plan B",
    "Push now push now",
    "Let me race please"
]
labels = [
    "진입 명령", "차량 상태", "불만/요청", "전략/전술", "차량 상태",
    "격려", "차량 상태", "전략/전술", "전략/전술", "불만/요청"
]